<a href="https://colab.research.google.com/github/cagutierrezgu/My_Portfolio/blob/main/Podcast%20topic%20classification/Data%20extraction/Data_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data loading

A continuación se presenta el desarrollo de un proyecto que busca analizar conversaciones provenientes de podcasts en inglés con diferentes temáticas. Las librerías a utilizar son las siguientes

In [ ]:
import requests
import matplotlib.pyplot as plt 
import numpy as np
from os import path, getcwd, listdir
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector
from wordcloud import WordCloud
from pprint import pprint

# Preprocesamiento y limpieza de datos
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec as w2v

# Modelos a implementar
import sklearn
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_samples
import gensim
from gensim import corpora
from gensim.models import CoherenceModel

El análisis se realizará sobre texto proveniente de conversaciones de varios capítulos de diferentes podcasts en inglés. Dicho material se encuentra alojado en la web, específicamente la información a utilizar se encuentra en la página de [happyscribe](https://www.happyscribe.com/), la cual almacena podcasts de diferentes temáticas con sus respectivos guiones.

En primer lugar, definimos algunas funciones para posteriormente extraer los textos a analizar:

In [ ]:
def get_soup(html):
    """Muestra toda la información de una página respectiva"""
    resp = requests.get(html)
    http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
    html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
    encoding = html_encoding or http_encoding
    soup = BeautifulSoup(resp.content, from_encoding=encoding)
    return soup

def get_links(soup):
    """Extrae los links contenidos en una página web"""
    http_link_list = [] 
    for link in soup.find_all('a', href=True):
        http_link_list.append(link['href'].strip("'"))
    return http_link_list

def get_ps(soup):
    """Obtiene los tags <p> de una página, que contienen el texto"""
    http_link_list = [] 
    for link in soup.find_all('p'):
        http_link_list.append(link.get_text())
    return http_link_list 

def get_text(text_array):
    """Junta el texto de una página web"""
    text = " ".join(text_array)
    return text

def get_episode_text(episode_list):
    """Lista el texto de cada episodio usando otras funciones ya definidas"""
    text_return = []
    for i in episode_list:
        #print(i)
        soup = get_soup(i)
        text_array = get_ps(soup)
        full_text = get_text(text_array)
        text_return.append(full_text)
    return text_return

El estudio se realizará sobre 5 diferentes podcasts, cada uno de temáticas diferentes. Uno de ellos será [Lex Fridman Podcast](https://www.happyscribe.com/public/lex-fridman-podcast-artificial-intelligence-ai) catalogado como un podcast de ciencia, así que cargarán alrededor de los últimos 100 episodios disponibles en la página web:

In [ ]:
science_transcripts = ['https://www.happyscribe.com/public/lex-fridman-podcast-artificial-intelligence-ai',
                      'https://www.happyscribe.com/public/lex-fridman-podcast-artificial-intelligence-ai?page=2',
                      'https://www.happyscribe.com/public/lex-fridman-podcast-artificial-intelligence-ai?page=3']

soup_science = list(map(get_soup, science_transcripts))

Una vez con toda la información de las páginas webs donde se encuentran los episodios a analizar se procede a extraer los links que estas contengan:

In [ ]:
h_links_science = list(map(get_links, soup_science))
links_science = []
for i in range(3):
    links_science += h_links_science[i]

Sin embargo, no todos los links son de interés para el proyecto, únicamente se requieren los que conducen a cada episodio del podcast, luego se filtran los deseados para el estudio:

In [ ]:
html_links_science = [lnk for lnk in links_science if 'ai/' in lnk]
#html_links_science

Ahora, teniendo los links requeridos se observa que todos estos muestran únicamente la extensión de cada uno de los capítulos, así que se completa la dirección de cada página web para luego cargar la información:

In [ ]:
html_links_science = ['https://www.happyscribe.com'+i for i in html_links_science]
#html_links_science

Una vez extraídos los links que contienen los guiones de los episodios a estudiar se carga el texto que cada uno de estos contiene, organizados en una lista cuyos elementos son cada uno de los episodios del podcast:

In [ ]:
text_science = get_episode_text(html_links_science)
#text_science

El procedimiento anterior es repetido para cargar el texto de los demás podcasts a estudiar. A continuación se carga uno con la temática de deportes llamado [The Dan Le Batard Show with Stugotz](https://www.happyscribe.com/public/the-dan-le-batard-show-with-stugotz):

In [ ]:
sports_transcripts = ['https://www.happyscribe.com/public/the-dan-le-batard-show-with-stugotz',
                     'https://www.happyscribe.com/public/the-dan-le-batard-show-with-stugotz?page=2',
                     'https://www.happyscribe.com/public/the-dan-le-batard-show-with-stugotz?page=3']

soup_sports = list(map(get_soup, sports_transcripts))

h_links_sports = list(map(get_links, soup_sports))

links_sports = []
for i in range(3):
    links_sports += h_links_sports[i]

html_links_sports = [lnk for lnk in links_sports if 'stugotz/' in lnk]
html_links_sports = ['https://www.happyscribe.com'+i for i in html_links_sports]

text_sports = get_episode_text(html_links_sports)
#text_sports

Ahora, uno catalogado como de salud y ejercicio, de nombre [The Mindset Mentor](https://www.happyscribe.com/public/the-mindset-mentor):

In [ ]:
hf_transcripts = ['https://www.happyscribe.com/public/the-mindset-mentor',
                 'https://www.happyscribe.com/public/the-mindset-mentor?page=2',
                 'https://www.happyscribe.com/public/the-mindset-mentor?page=3']

soup_hf = list(map(get_soup, hf_transcripts))

h_links_hf = list(map(get_links, soup_hf))

links_hf = []
for i in range(3):
    links_hf += h_links_hf[i]

html_links_hf = [lnk for lnk in links_hf if 'mentor/' in lnk]
html_links_hf = ['https://www.happyscribe.com'+i for i in html_links_hf]

text_hf = get_episode_text(html_links_hf)
#text_hf

Otro de los podcasts a utilizar es uno llamado [Stuff You Missed in History Class](https://www.happyscribe.com/public/stuff-you-missed-in-history-class), cuya temática es de historia:

In [ ]:
history_transcripts = ['https://www.happyscribe.com/public/stuff-you-missed-in-history-class',
                      'https://www.happyscribe.com/public/stuff-you-missed-in-history-class?page=2',
                      'https://www.happyscribe.com/public/stuff-you-missed-in-history-class?page=3']

soup_history = list(map(get_soup, history_transcripts))

h_links_history = list(map(get_links, soup_history))

links_history = []
for i in range(3):
    links_history += h_links_history[i]

html_links_history = [lnk for lnk in links_history if 'class/' in lnk]
html_links_history = ['https://www.happyscribe.com'+i for i in html_links_history]

text_history = get_episode_text(html_links_history)
#text_history

Finalmente, el último de los podcasts se llama [My Favorite Murder with Karen Kilgariff and Georgia Hardstark](https://www.happyscribe.com/public/my-favorite-murder-with-karen-kilgariff-and-georgia-hardstark), considerado como de crimen:

In [ ]:
crime_transcripts = ['https://www.happyscribe.com/public/my-favorite-murder-with-karen-kilgariff-and-georgia-hardstark',
                    'https://www.happyscribe.com/public/my-favorite-murder-with-karen-kilgariff-and-georgia-hardstark?page=2',
                    'https://www.happyscribe.com/public/my-favorite-murder-with-karen-kilgariff-and-georgia-hardstark?page=3']

soup_crime = list(map(get_soup, crime_transcripts))

h_links_crime = list(map(get_links, soup_crime))

links_crime = []
for i in range(3):
    links_crime += h_links_crime[i]

html_links_crime = [lnk for lnk in links_crime if 'hardstark/' in lnk]
html_links_crime = ['https://www.happyscribe.com'+i for i in html_links_crime]

text_crime = get_episode_text(html_links_crime)
#text_crime

Al verificar el número de episodios que contiene cada uno de los podcasts cargados se observa que cada uno contiene 60 episiodios:

In [ ]:
print(len(text_crime), len(text_science), len(text_history), len(text_hf), len(text_sports))

60 60 60 60 60


Es decir, se trabajará con 300 conjuntos de texto, en principio independientes.